#### This notebook processes and analyzes monthly vertical fluxes data (rld, rldcs, rlu, rlucs) from Atmospheric Model Intercomparison Project (AMIP) simulations to compute vertical profiles of Cloud Radiative Heating rate (CRH, K/day) across multiple climate models.

import packages

In [2]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import xarray as xr
import pickle
import os
import time
import glob
from utils import height2pressure

Calculating ACRE at each level from vertical fluxes using Eq. (3) in the supplement materials of Liu and Grise (2025)

In [13]:
model_names = ['BCC-CSM2-MR','INM-CM4-8','INM-CM5-0','MIROC6','MPI-ESM-1-2-HAM','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0']
directory='/OWC/huiyu/CMIP6/vertical_CRH'
for model_name in model_names:
    print(model_name)
    
    # Use glob to find the files matching the pattern
    rlu_file = glob.glob(os.path.join(directory, 'combined', f'rlu_CFmon_{model_name}_amip_*.nc'))[0]
    rlucs_file = glob.glob(os.path.join(directory, 'combined', f'rlucs_CFmon_{model_name}_amip_*.nc'))[0]
    rld_file = glob.glob(os.path.join(directory, 'combined', f'rld_CFmon_{model_name}_amip_*.nc'))[0]
    rldcs_file = glob.glob(os.path.join(directory, 'combined', f'rldcs_CFmon_{model_name}_amip_*.nc'))[0]

    rlu = xr.open_dataset(rlu_file)
    rlucs = xr.open_dataset(rlucs_file)
    rld = xr.open_dataset(rld_file)
    rldcs = xr.open_dataset(rldcs_file)

    acre = rld["rld"] - rldcs["rldcs"] - rlu["rlu"] + rlucs["rlucs"]

    acre_ds = rld.copy()
    acre_ds["acre"] = acre
    acre_ds = acre_ds.drop_vars(["rld"])
    acre_ds.to_netcdf(os.path.join(directory, 'combined', f'combined_fluxes_{model_name}.nc'))

MRI-ESM2-0


Define functions

In [4]:
def identify_model_level_type(nc_file):
    """
    Identify the model vertical level type from a NetCDF file.
    
    Parameters:
    -----------
    nc_file : str
        Path to the NetCDF file to be opened.

    Returns:
    --------
    tuple
        A tuple containing the standard name and units of the vertical level variable.
        Returns values for one of 'lev', 'presnivs', or 'plev' if found.
    """
    ds = xr.open_dataset(nc_file)
    if 'lev' in ds.variables:
        return ds.lev.standard_name
    elif 'presnivs' in ds.variables:
        return ds.presnivs.standard_name
    elif 'plev' in ds.variables:
        return ds.plev.standard_name

In [3]:
def define_targetlevels():
    """ Defines the pressure levels for the interpolation. """
    import numpy as _np
    return _np.arange(1000e2, 0, -25e2)

In [21]:
def height_to_pressure(ds):
    """
    Converts model height coordinates (unit: m) to pressure coordinates,
    computes zonal mean, removes invalid levels (with NaNs),
    and interpolates onto standard pressure levels.

    Parameters:
    -----------
    filename : str
        Path to the NetCDF file containing a dataset with height-based 'lev' coordinate.

    Returns:
    --------
    xarray.Dataset
        Dataset with 'lev' now representing pressure levels and interpolated to standard levels.
    """
    #ds = xr.open_dataset(filename)
    pressure_lev = height2pressure(ds.lev.values)
    ds = ds.assign_coords(pressure_lev=("lev", pressure_lev)).swap_dims({"lev":"pressure_lev"}).drop_vars("lev")
    ds = ds.rename({"pressure_lev": "lev"})
    zonal_mean_ds = ds.mean(dim="lon")
    # Remove levels with NaNs
    valid_lev = zonal_mean_ds['lev'].where(~np.isnan(zonal_mean_ds['lev']), drop=True)
    indices = [i for i, lev_value in enumerate(zonal_mean_ds['lev'].values) if lev_value in valid_lev]
    zonal_mean_ds = zonal_mean_ds.isel(lev=indices)
    # Interpolate to standard pressure levels
    zonal_mean_ds_interped = zonal_mean_ds.interp(lev=define_targetlevels())
    return zonal_mean_ds_interped

In [22]:
def hybrid_height_to_pressure(ds):
    """
    Converts model hybrid height coordinates (unit: 1) to pressure coordinates,
    computes zonal mean, removes invalid levels (with NaNs),
    and interpolates onto standard pressure levels.

    Parameters:
    -----------
    filename : str
        Path to the NetCDF file containing a dataset with hybrid-height-based 'lev' coordinate.

    Returns:
    --------
    xarray.Dataset
        Dataset with 'lev' now representing pressure levels and interpolated to standard levels.
    """
    #ds = xr.open_dataset(filename)
    b = ds['b'].values.reshape(-1, 1, 1)  # Reference pressure (scalar)
    orog = ds['orog'].values
    sigma = ds['lev'].values.reshape(-1, 1, 1)  # Reshape to (lev, 1, 1)
    height_levels = sigma + b*orog[np.newaxis, :, :]
    pressure_lev = height2pressure(height_levels)
    ds = ds.assign_coords(pressure_lev=("lev", pressure_lev)).swap_dims({"lev":"pressure_lev"}).drop_vars("lev")
    ds = ds.rename({"pressure_lev": "lev"})
    zonal_mean_ds = ds.mean(dim="lon")
    # Remove levels with NaNs
    valid_lev = zonal_mean_ds['lev'].where(~np.isnan(zonal_mean_ds['lev']), drop=True)
    indices = [i for i, lev_value in enumerate(zonal_mean_ds['lev'].values) if lev_value in valid_lev]
    zonal_mean_ds = zonal_mean_ds.isel(lev=indices)
    # Interpolate to standard pressure levels
    zonal_mean_ds_interped = zonal_mean_ds.interp(lev=define_targetlevels())
    return zonal_mean_ds_interped

In [23]:
def model_level_to_pressure(ds):
    """
    Converts model "atmospheric_hybrid_sigma_pressure_coordiante" (unit: 1) or "atmospheric_hybrid_sigma_coordiante" to pressure coordinates,
    computes zonal mean, removes invalid levels (with NaNs),
    and interpolates onto standard pressure levels.

    Parameters:
    -----------
    filename : str
        Path to the NetCDF file containing a dataset with height-based 'lev' coordinate.

    Returns:
    --------
    xarray.Dataset
        Dataset with 'lev' now representing pressure levels and interpolated to standard levels.
    """

    if 'ap' in ds.variables:
        ap = ds['ap'].values.reshape(-1, 1, 1)
        b = ds['b'].values.reshape(-1, 1, 1)  # Reshape to (lev, 1, 1)
        ps = ds['ps'].values
        pressure_levels = ap + b * ps[:, np.newaxis, :, :]  
    else:
        p0 = ds['p0'].values  # Reference pressure (scalar)
        a = ds['a'].values.reshape(-1, 1, 1)  # Reshape to (lev, 1, 1)
        b = ds['b'].values.reshape(-1, 1, 1)  # Reshape to (lev, 1, 1)
        ps = ds['ps'].values
        pressure_levels = a * p0 + b * ps[:, np.newaxis, :, :]
    averaged_pressure_lev = np.mean(pressure_levels, axis=(0, 2, 3))
    ds = ds.assign_coords(pressure_lev=("lev", averaged_pressure_lev)).swap_dims({"lev":"pressure_lev"}).drop_vars("lev")
    ds = ds.rename({"pressure_lev": "lev"})
    zonal_mean_ds = ds.mean(dim="lon")
    zonal_mean_ds_interped = zonal_mean_ds.interp(lev=define_targetlevels())
    return zonal_mean_ds_interped

Calculating CRH from ACRE using Eq. 8 in the supplementary materials of Liu and Grise (2025)

In [ ]:
directory='/OWC/huiyu/CMIP6/vertical_CRH'
# Constants
g = 9.81  # gravitational acceleration in m/s^2
cp = 1004  # specific heat capacity of air at constant pressure in J/kg/K
seconds_in_day = 86400  # seconds in a day
model_names = ['BCC-CSM2-MR','INM-CM4-8','INM-CM5-0','MIROC6','MPI-ESM-1-2-HAM','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0']
for model_name in model_names:
    print(model_name)
    filename = directory+'/combined/combined_fluxes_'+model_name+'.nc'
    ds = xr.open_dataset(filename, chunks={'lev': 10})
    # Compute tendencies lazily
    temp_tendency = -g / cp * ds['acre'].diff(dim="lev") / ds['lev'].diff(dim="lev")
    temp_tendency_K_day = temp_tendency * seconds_in_day
    ds['temp_tendency_K_day'] = temp_tendency_K_day
    # Identify the model level type
    level_type = identify_model_level_type(filename)
    print("Level type:", level_type)
    if level_type == 'atmosphere_hybrid_height_coordinate':
        ds_pressure = hybrid_height_to_pressure(ds)
    elif level_type == 'atmosphere_height_coordinate':
        ds_pressure = height_to_pressure(ds)
    elif level_type == 'atmosphere_hybrid_sigma_pressure_coordinate':
        ds_pressure = model_level_to_pressure(ds)
    elif level_type == 'atmosphere_sigma_coordinate':
        ds_pressure = model_level_to_pressure(ds)
    else:
        raise ValueError("Unknown model level type")
    
    ds_interpolated = ds_pressure.interp(lev=define_targetlevels()).compute()
    # Save the dataset
    ds_interpolated.to_netcdf(directory+'/interpolated/'+model_name+'_CRH_interpolated_zonalmean.nc')